NOMBRE: Jose Miguel Gonzalez


CARNE: 20335

FECHA: 9/29/2024

**Comentario General:** El objetivo de esta práctica es que exploren la arquitectura transformer que ya debieron haber leído en el paper original.

Cuando lleguen a la parte de entrenamiento y evaluación, procuren obtener el mejor resultado para WER. Cambien parametros, congelen capas convolucionales, etc. Dejen evidencia de todos sus intentos.

Responda las siguientes preguntas después de haber leído el paper original de Wav2Vec2(Se encuentra en: Materiales del profesor/papers/transformers y LLM):
¿Qué es Wav2Vec2 y qué problema intenta resolver?
¿Cómo se diferencia Wav2Vec2 de los métodos tradicionales de reconocimiento de voz?
¿Cuáles son los componentes principales de la arquitectura de Wav2Vec2?
¿Puedes explicar el proceso de entrenamiento de Wav2Vec2? ¿Cómo funciona el aprendizaje auto-supervisado en este contexto?
¿Qué papel juega la cuantización de características de audio en Wav2Vec2?
¿Cuáles son las ventajas y desventajas de usar Wav2Vec2 en comparación con otros modelos de reconocimiento de voz?

In [ ]:
#Librerias a instalar, en caso de no estar instaladas

#!pip install datasets
#!pip install transformers
#!pip install jiwer

In [ ]:
#Dataset con el que estarán trabajando
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

In [ ]:
#Elimine las columnas innecesarias(para este caso didactico). Solo necesita "text","audio","file" y "id".

timit = timit.remove_columns([col for col in timit["train"].column_names if col not in ["text", "audio", "file", "id"]])


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd

#defina una función que reciba el dataset y in int "n". La función debe mostrar un dataframe "n" textos del dataset.

def mostrar_textos(dataset, n):
    df = pd.DataFrame(dataset["train"]["text"])
    return df.sample(n)


In [ ]:
import re

#Limpie los textos.
def limpiar_texto(texto):
    texto = re.sub(r"[^a-zA-Z\s]", "", texto)  # Eliminar caracteres especiales
    texto = texto.lower()  # Convertir a minúsculas
    return texto
timit = timit.map(lambda x: {"text": limpiar_texto(x["text"])})


In [ ]:
#Defina una funcion para construir el vocabulario. Esto es todos los caracteres que están en nuestro corpus.
#Ejemplo: {a: 3, b:5, c:7}
#Luego, cambie el espacio " " por "|". Esto es una buena practica, para tener un elemento visible para el espacio.
#Al vocabulario debe agregar "[UNK]" y "[PAD]". Uno es para desconocidos, otro es para en blanco. Estos deben tener asignada
#la longitud del vocabulario.
#Guarde el vocabulario en un json: vocab.json

def construir_vocabulario(dataset):
    vocabulario = {}
    for texto in dataset["train"]["text"]:
        for char in texto:
            if char not in vocabulario:
                vocabulario[char] = 1
            else:
                vocabulario[char] += 1
    vocabulario["|"] = vocabulario.get(" ", 0)  # Reemplaza el espacio con "|"
    vocabulario["[UNK]"] = len(vocabulario) + 1
    vocabulario["[PAD]"] = len(vocabulario) + 1
    return vocabulario

vocabulario = construir_vocabulario(timit)

import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocabulario, vocab_file)


In [ ]:
from transformers import Wav2Vec2CTCTokenizer
#Use el vocabulario anterior para tokenizar con Wav2Vec2TCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")


In [ ]:
from transformers import Wav2Vec2FeatureExtractor
#Defina el pipeline de extraccion de caracteristicas

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding=True)


In [ ]:
from transformers import Wav2Vec2Processor
#Defina el pipeline de procesamiento

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)


In [ ]:
#Antes de empezar, evalue que todo esta bien: escuche un par de audios y vea si el texto de referencia es correcto






In [ ]:
#Use la siguiente funcion para crear los batches en el formato que el modelo necesita
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
#Training enviroment: use este como guía, puede que varíe respecto a como nombraron sus variables anteriormente.
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
#Metrica de rendimiento WER
wer_metric = load_metric("wer")
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base",
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
)

In [ ]:
#Parametros de entrenamiento segun la documentacion

from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir=repo_name,
  group_by_length=True,
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit["train"],
    eval_dataset=timit["test"],
    tokenizer=processor.feature_extractor,
)

In [ ]:
timit = timit.map(prepare_dataset, remove_columns=timit.column_names, batched=True)


In [ ]:
repo_name = "wav2vec2_asr_timit"


In [ ]:
trainer.train()
